# DSCI Group Project Proposal

_Group 66 (Andrew Ahn, Calvin Choi, Allan Hu, Dishika Taneja_)

### Introduction ###

### Abstract (Preliminary exploratory data analysis)

### Methods

### Hypothesis (Expected outcomes and significance)